In [2]:
import sys, os; sys.path.append(os.path.dirname(os.getcwd())) 

import matplotlib.pyplot as plt
# from ipywidgets import *

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import check_is_fitted
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import GridSearchCV
# from sklearn import neighbors, clone


from pyfrechet.metric_spaces import MetricData, Sphere, Euclidean
# from pyfrechet.regression.frechet_regression import LocalFrechet, GlobalFrechet
# from pyfrechet.regression.kernels import NadarayaWatson, gaussian, epanechnikov
# from pyfrechet.regression.knn import KNearestNeighbours
from pyfrechet.regression.bagged_regressor import BaggedRegressor
from pyfrechet.regression.trees import Tree
from pyfrechet.metrics import mse

In [3]:

M = Sphere(2)
m = lambda x: 5 + 5*x[:,0]**2 + np.sin(20 * x[:,0]) - 10*x[:,0]**3 

def gen_data(N, eps=0.1):
    # Univariate predictor in [0,1]
    x = np.random.rand(N*1).reshape((N,1))
    theta = m(x) + eps*np.random.randn(N)
    y = np.c_[np.cos(theta), np.sin(theta)]
    return x, MetricData(M, y)

N = 500; eps = 0.5
X, y = gen_data(N, eps)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns


grid=np.linspace(0,1,100)
grid_m=5 + 5*grid**2 + np.sin(20 * grid) - 10*grid**3
grid_y=np.c_[np.cos(grid_m), np.sin(grid_m)]

fig=plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(grid_y, grid_y[:,0], grid_m[:,1])
plt.show()

AttributeError: module 'matplotlib.pyplot' has no attribute 'ax_scatter'

In [21]:
grid_m.shape

NameError: name 'grid_m' is not defined

In [5]:
Base = Tree(split_type='2means',
            impurity_method='cart',
            mtry=None,
            min_split_size=5)
Forest = BaggedRegressor(estimator=Base,
                         n_estimators=100,
                         bootstrap_fraction=1,
                         bootstrap_replace=True,
                         n_jobs=-2)
Forest.fit(X_train, y_train)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:  3.0min finished


BaggedRegressor(bootstrap_fraction=1, bootstrap_replace=True,
                estimator=Tree(split_type='2means'))

In [6]:
Forest_predictions = Forest.predict(X_test)

In [7]:
print(f'MSE: {mse(y_test, Forest_predictions)}')

MSE: 0.276405849698628


In [8]:
# Checking if bootstrap samples with replacement are correct
print(X_train.shape)
print(Forest.estimators[0][0].shape)
print(np.unique(Forest.estimators[0][0], return_counts=True))
print(np.unique(Forest.estimators[0][0], return_counts=False).shape)

(400, 1)
(400,)
(array([  0,   1,   2,   3,   5,   7,   8,  11,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  31,  34,
        35,  36,  38,  39,  40,  42,  43,  45,  46,  47,  48,  49,  52,
        53,  55,  56,  58,  62,  63,  65,  66,  67,  70,  71,  72,  73,
        75,  77,  78,  80,  82,  83,  84,  85,  86,  87,  88,  90,  92,
        93,  95,  97,  98,  99, 101, 105, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 122, 123, 125, 128, 129, 131,
       132, 135, 136, 137, 140, 143, 144, 145, 146, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 163, 164, 166, 169, 170,
       171, 172, 173, 175, 176, 177, 178, 179, 183, 184, 185, 189, 190,
       194, 196, 197, 198, 199, 200, 202, 203, 205, 206, 208, 209, 212,
       213, 214, 215, 216, 218, 219, 223, 225, 226, 227, 228, 229, 230,
       232, 233, 237, 238, 239, 240, 242, 246, 250, 254, 255, 256, 257,
       261, 263, 264, 265, 269, 271, 273, 274, 

In [9]:
type(Forest.estimators[0][0])

numpy.ndarray

In [10]:
Forest._oob_predict_one(X_train[10].reshape(-1,1))

array([-0.58685514,  0.80969195])

In [11]:
prueba=Forest.oob_predict(X_train)
prueba.data

array([[ 1.44736633e-01, -9.89470215e-01],
       [ 9.99698977e-01, -2.45347826e-02],
       [-9.94762252e-01,  1.02215763e-01],
       [ 9.94400647e-01, -1.05675698e-01],
       [-6.27250861e-01, -7.78817281e-01],
       [-3.46109459e-01, -9.38194139e-01],
       [ 8.41675018e-01, -5.39984411e-01],
       [ 1.99759624e-01, -9.79844933e-01],
       [-2.55246916e-01, -9.66875903e-01],
       [ 8.90749846e-01, -4.54493907e-01],
       [-5.86855136e-01,  8.09691948e-01],
       [ 1.56778661e-01,  9.87633764e-01],
       [ 9.08679724e-01, -4.17493903e-01],
       [-5.50844765e-01, -8.34607719e-01],
       [ 7.17369715e-01,  6.96692681e-01],
       [-5.64906576e-01, -8.25154871e-01],
       [ 5.85517685e-01, -8.10659633e-01],
       [ 9.99961319e-01, -8.79545223e-03],
       [-5.70857530e-01, -8.21049134e-01],
       [ 7.63730987e-01, -6.45534646e-01],
       [ 8.36894951e-01, -5.47363536e-01],
       [ 6.11016470e-01,  7.91617883e-01],
       [ 5.25610992e-01, -8.50725035e-01],
       [ 9.

In [13]:
# OOB "prediction error (distance)"
print(prueba.M.d(prueba.data, y_train.data).shape)
(prueba.M.d(prueba.data, y_train.data)**2).mean()

(400,)


0.2731211647021828

In [14]:
type(y_train)

pyfrechet.metric_spaces.metric_data.MetricData

In [15]:
# OOB prediction errors
Forest.oob_errors()

array([7.33005940e-01, 2.05719650e-01, 1.32787620e-01, 1.49800230e-01,
       6.24757111e-01, 3.77984090e-01, 3.32070420e-02, 6.60585839e-01,
       6.82818523e-02, 1.74774033e-01, 8.49952225e-02, 1.18885301e-01,
       5.02792192e-01, 1.09212672e-01, 3.08549668e-01, 8.14320185e-01,
       1.73777046e-01, 4.69668759e-01, 1.18991770e-01, 3.56518982e-01,
       5.98910050e-01, 8.70520151e-02, 8.65701977e-01, 5.18655792e-01,
       7.59183365e-01, 4.48517332e-02, 3.83101326e-01, 8.49183250e-01,
       4.30824654e-01, 9.21229802e-02, 5.78906997e-01, 3.56294576e-01,
       4.16978284e-01, 9.24090721e-01, 1.23114033e-01, 1.15951445e+00,
       7.43496742e-01, 3.99367999e-01, 2.67320024e-01, 1.15732244e-01,
       4.97287166e-01, 8.92577475e-02, 1.96552824e-03, 7.70009013e-01,
       3.25041852e-01, 2.99609391e-01, 6.51427571e-01, 3.61565617e-01,
       3.82739494e-01, 2.21098827e-01, 2.23480802e-01, 4.32033526e-01,
       3.12780258e-02, 7.21372728e-01, 5.91372869e-01, 3.86985644e-01,
      

In [16]:
# Quantile for confidence balls
alpha=0.05
Dalpha=np.percentile(Forest.oob_errors(), (1-alpha)*100)

In [17]:
np.mean(y_train.M.d(Forest_predictions.data, y_test.data) <= Dalpha)

0.97

In [12]:
type(y_train)

pyfrechet.metric_spaces.metric_data.MetricData